In [ ]:
#avoid TF to consume GPU memory
import os
import glob
import tensorflow as tf
import mdtraj as md
import matplotlib.pyplot as plt
import gromacs as gmx 
import numpy as np
import nglview as nv
import ASMSAnalysis 

tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

#%matplotlib widget


In [ ]:
conf = "../../Thermal-unfolding/trpcage_npt400_nH.pdb"
traj = "../../Thermal-unfolding/trpcage_ds_nH.xtc"
hills = "../4.MD/mtd1/HILLS-1"
lows_filename = "../3.Train/low/lows.txt"

analysis = ASMSAnalysis.Analysis(conf,traj,hills,lows_filename)

In [ ]:

conf_nh, traj_fit = analysis.prepare_trajectory()
tr = md.load(traj_fit, top=conf_nh)

idx_res = tr.top.select("residue 6")
idx_b = tr.top.select("backbone")
idx_ca = tr.top.select("name CA")
idx = tr.top.select("protein")

tr.superpose(tr, atom_indices=idx_b)

rmsds = md.rmsd(tr, tr, 0, atom_indices=idx_res, precentered=True)

plt.figure(figsize=(10, 4))
plt.title(f'RMSD Backbone - {int(tr.time[-1]/1000)} ns')
plt.scatter(tr.time / 1000, rmsds, c=rmsds, s=0.1, cmap='jet')
plt.xlabel('Time (ns)')
plt.ylabel('RMSD (nm)')
plt.colorbar(label='')
plt.show()

In [ ]:
analysis.on_the_flight(interval=30)

In [ ]:
conf_nh, traj_fit = analysis.prepare_trajectory()
tr = md.load(traj_fit, top=conf_nh)

directory = './'  
files_to_delete = glob.glob(os.path.join(directory, '#*'))
for file in files_to_delete:
    os.remove(file)



v = nv.show_mdtraj(tr, default=False, gui=True)  # Enable GUI for play button
v.add_cartoon(selection="protein", color='red')
c = v.add_component(tr[0], default=False, gui=False)
c.add_cartoon(selection="protein", color='blue')

v.add_licorice(selection="res 6", c='red')
c.add_licorice(selection="res 6", c='blue')


v.center(selection='protein')
v.render_image(trim=True, factor=3)
v.camera = 'orthographic'
v.background = 'white'
v

In [ ]:
gmx.editconf(f='npt.gro',input='Protein-H',o='npt_nh.pdb', ndef=True)

In [ ]:
analysis.calculate_rmsd()

In [ ]:
analysis.highlights_and_dynamic()

In [ ]:
directory = './'  
files_to_delete = glob.glob(os.path.join(directory, '#*'))
for file in files_to_delete:
    os.remove(file)

In [ ]:
!pip install metadynminer

In [ ]:
import metadynminer as m

In [ ]:
hf = m.Hills(name="HILLS")

In [ ]:
fes = m.Fes(hf)

In [ ]:
fes.plot()

In [ ]:
minima = m.Minima(fes)
print(minima.minima)
minima.plot()

In [ ]:
fep = metadynminer.FEProfile(minima, hillsfile)
fep.plot()